# Tutorial 2: Finetuning Bert for Sequence Classification using a LoRA adapter

When we import a pretrained transformer model from HuggingFace, we receive the encoder/decoder weights, which aren't that useful on their own - to perform a useful task such as sequence classification, we add a classification head on top of the model and train those weights on the required dataset. In this tutorial, we'll look at fine tuning a Bert model for sequence classification with two approaches. First, we'll attempt full Supervised Fine Tuning (SFT). Then, we'll use the Mase stack to add a [LoRA](https://arxiv.org/abs/2106.09685) adapter to the model. We'll look at the effect in memory requirement for training and the achieved accuracy.

## Sentiment Analysis with the IMDb Dataset

The IMDB dataset, introduced in [this 2011 paper](https://aclanthology.org/P11-1015/) from Stanford, is commonly used for sentiment analysis in the Natural Language Processing (NLP) community. This is a collection of 50k movie reviews from the IMDb website, labelled as either "positive" or "negative". Here is an example of a positive review:

> I turned over to this film in the middle of the night and very nearly skipped right passed it. It was only because there was nothing else on that I decided to watch it. In the end, I thought it was great.<br /><br />An interesting storyline, good characters, a clever script and brilliant directing makes this a fine film to sit down and watch. This was, in fact, the first I'd heard of this movie, but I would have been happy to have paid money to see this at the cinema.<br /><br />My IMDB Rating : 8 out of 10<br /><br />

And a negative review:

> its a totally average film with a few semi-alright action sequences that make the plot seem a little better and remind the viewer of the classic van dam films. parts of the plot don't make sense and seem to be added in to use up time. the end plot is that of a very basic type that doesn't leave the viewer guessing and any twists are obvious from the beginning. the end scene with the flask backs don't make sense as they are added in and seem to have little relevance to the history of van dam's character. not really worth watching again, bit disappointed in the end production, even though it is apparent it was shot on a low budget certain shots and sections in the film are of poor directed quality

The dataset is available from HuggingFace through the ``datasets`` library. We use the `get_tokenized_dataset` utility in Mase to automatically tokenize it.

In [ ]:
from chop.tools import get_tokenized_dataset

dataset, tokenizer = get_tokenized_dataset(
    dataset="imdb",
    checkpoint="bert-base-uncased",
    return_tokenizer=True,
)

## Generate a MaseGraph with Custom Arguments

By inspecting the implementation of the Bert model in HuggingFace, we can see the forward function has a signature similar to the following.

```python
    class BertForSequenceClassification(BertPreTrainedModel):
        def __init__(self, config):
            super().__init__(config)
            self.bert = BertModel(config)
            ...

        def forward(
            self,
            input_ids: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            token_type_ids: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.Tensor] = None,
            head_mask: Optional[torch.Tensor] = None,
            inputs_embeds: Optional[torch.Tensor] = None,
            labels: Optional[torch.Tensor] = None,
            output_attentions: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
        ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
            ...
```

By default, the MaseGraph constructor chooses to use the `input_ids` argument, ignoring the other optional arguments. However, you can specify which inputs to drive during symbolic tracing using the `hf_input_names` argument. In the following cell, we also drive the `attention_mask` and `labels` inputs. By specifying the `labels` argument, we include a `nn.CrossEntropyLoss` module at the end of the model to calculate the loss directly.

> **Task:** Remove the `attention_mask` and `labels` arguments from the `hf_input_names` list and re-run the following cell. Use `mg.draw()` to visualize the graph in each case. Can you see any changes in the graph topology? Can you explain why this happens?

In [ ]:
from transformers import AutoModelForSequenceClassification

from chop import MaseGraph
import chop.passes as passes

checkpoint = "prajjwal1/bert-tiny"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.config.problem_type = "single_label_classification"

mg = MaseGraph(
    model,
    hf_input_names=[
        "input_ids",
        "attention_mask",
        "labels",
    ],
)

mg, _ = passes.init_metadata_analysis_pass(mg)
mg, _ = passes.add_common_metadata_analysis_pass(
    mg,
    pass_args={
        "add_value": False,
    },
)

## Full Supervised Finetuning (SFT)

Before training the model, let's inspect how many trainable parameters there are. If you're familiar with Keras, you might have used the `model.summary()` API before, but it's not as easy to do the same in Pytorch - luckily, Mase has a module-level pass with this functionality.

In [ ]:
from chop.passes.module import report_trainable_parameters_analysis_pass

mg, _ = report_trainable_parameters_analysis_pass(mg.model)

From this, we can see the majority of the trainable parameters are in the `Embedding` layer. We don't need to train this, so we freeze those parameters in the cell below.

In [ ]:
for param in mg.model.bert.embeddings.parameters():
    param.requires_grad = False

To train the model, we rely on the `Trainer` class from the `transformers` library, which makes it easy to set up a training loop with any hardware configuration. The `get_trainer` utility in Mase handles assigning the training arguments to the `Trainer` class for common use cases, such as in this tutorial.

In [ ]:
from chop.tools import get_trainer

trainer = get_trainer(
    model=mg.model,
    tokenized_dataset=dataset,
    tokenizer=tokenizer,
    evaluate_metric="accuracy",
)

Before running any fine tuning, let's see how the model performs out of the box. Without any fine-tuning, we can see the model just performs a random guess - there are two labels in the dataset, so this corresponds to an accuracy of around 50%.

In [ ]:
# Evaluate accuracy
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

Now, run the cell below to execute a single training epoch with the current setup.

In [ ]:
trainer.train()

Let's see how much accuracy we get after a single training epoch of full finetuning.

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

We can now export the SFT version of the model to be used in later tutorials.

In [ ]:
from pathlib import Path

mg.export(f"{Path.home()}/tutorial_2_sft")

## Parameter Efficient Finetuning (PEFT) with LoRA

An alternative to full fine-tuning is Parameter Efficient Fine Tuning (PEFT), which uses a small number of trainable parameters to achieve similar performance. LoRA was proposed by a research team at Microsoft in 2021, as an efficient technique for PEFT. 

<div style="text-align: center;">
    <img src="imgs/lora_adapter.png" alt="drawing" width="400"/>
</div>

Consider the standard equation of a linear layer:

$$
y = X W + b
$$

The LoRA method involves replacing this with the following, where A and B are low-rank matrices. We freeze the $W$ parameters, and only allow the optimizer to train the parameters in $A$ and $B$.

$$
y = X (W + AB) + b
$$

This enables us to achieve accuracies comparable to full fine tuning, while only training a fraction of the parameters. See [the paper](https://arxiv.org/abs/2106.09685) for more details. We can inject the LoRA adapter into the existing model using the `insert_lora_adapter_transform_pass` pass in Mase, as follows.

In [ ]:
mg, _ = passes.insert_lora_adapter_transform_pass(
    mg,
    pass_args={
        "rank": 6,
        "alpha": 1.0,
        "dropout": 0.5,
    },
)

Similar to before, let's report the number of trainable parameters.

In [ ]:
mg, _ = report_trainable_parameters_analysis_pass(mg.model)

In this case, LoRA reduces the number of trainable parameters by $11.6\times$! We'll run a few more training epochs and evaluate the resulting accuracy.

In [ ]:
trainer = get_trainer(
    model=mg.model,
    tokenized_dataset=dataset,
    tokenizer=tokenizer,
    evaluate_metric="accuracy",
    num_train_epochs=1,
)
trainer.train()

# Evaluate accuracy
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

After training is finished, we can run the `fuse_lora_weights_transform_pass` pass to optimize the model for inference. This pass replaces each `LoRALinear` instance with an `nn.Linear` module, where the $AB$ product added to the original weights matrix. This incurs less kernel invocations when deploying the model, which reduces inference runtime.

In [ ]:
mg, _ = passes.fuse_lora_weights_transform_pass(mg)

## Conclusion

Finally, export the finetuned model to be used in future tutorials.

In [ ]:
from pathlib import Path

mg.export(f"{Path.home()}/tutorial_2_lora")